In [ ]:
import pandas as pd
df=pd.read_csv('/content/Data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
columns = [
    "Sample_code_number", "Clump_Thickness", "Uniformity_Cell_Size",
    "Uniformity_Cell_Shape", "Marginal_Adhesion", "Single_Epithelial_Cell_Size",
    "Bare_Nuclei", "Bland_Chromatin", "Normal_Nucleoli", "Mitoses", "Class"
]
data = pd.read_csv(url, names=columns)
data['Bare_Nuclei'] = pd.to_numeric(data['Bare_Nuclei'], errors='coerce')
data.dropna(inplace=True)
X = data.iloc[:, 1:-1]
y = data['Class'].apply(lambda x: 1 if x == 4 else 0)

In [ ]:
# Split and scale the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define models and hyperparameter grids
models = {
    "Logistic Regression": (LogisticRegression(), {"C": [0.01, 0.1, 1, 10]}),
    "Random Forest": (RandomForestClassifier(), {"n_estimators": [50, 100, 200], "max_depth": [None, 10, 20]}),
    "SVM": (SVC(), {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"], "gamma": ["scale", "auto"]}),
    "Gradient Boosting": (GradientBoostingClassifier(), {"learning_rate": [0.01, 0.1, 0.2], "n_estimators": [50, 100]}),
    "KNN": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7], "weights": ["uniform", "distance"], "p": [1, 2]})
}


In [ ]:
# Grid Search CV for each model
best_models = {}
for model_name, (model, params) in models.items():
    grid = GridSearchCV(model, params, scoring="f1", cv=5)
    grid.fit(X_train, y_train)
    best_models[model_name] = grid.best_estimator_
    print(f"{model_name} Best Parameters: {grid.best_params_}")

Logistic Regression Best Parameters: {'C': 0.1}
Random Forest Best Parameters: {'max_depth': 20, 'n_estimators': 200}
SVM Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Gradient Boosting Best Parameters: {'learning_rate': 0.01, 'n_estimators': 100}
KNN Best Parameters: {'n_neighbors': 7, 'p': 1, 'weights': 'uniform'}


In [ ]:
# Evaluate the best models
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"\n{model_name} Classification Report:\n")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else y_pred)}")



Logistic Regression Classification Report:

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       127
           1       0.97      0.88      0.93        78

    accuracy                           0.95       205
   macro avg       0.95      0.93      0.94       205
weighted avg       0.95      0.95      0.95       205

ROC-AUC Score: 0.9969715324046033

Random Forest Classification Report:

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       127
           1       0.97      0.94      0.95        78

    accuracy                           0.97       205
   macro avg       0.97      0.96      0.96       205
weighted avg       0.97      0.97      0.97       205

ROC-AUC Score: 0.9957601453664446

SVM Classification Report:

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       127
           1       0.96      0.94      0.95        78

 